# Artificial Persuasive Intelligence

In [2]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [6]:
cd persuasio.staging


/mnt/custom-file-systems/efs/fs-0efe0723c8fe23def_fsap-009815092600551f0/projects/persuasio.staging


## Libraries

In [7]:
from datasets import load_dataset, Value, Sequence, Features
import chromadb
import pandas as pd
import re
import numpy as np 
import chuck_gpt
import concurrent
import boto3
from datetime import datetime as dt
import inspect

from importlib import reload
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Matern, DotProduct
from scipy.stats import ecdf, norm
import random
import requests
import json
import chuck_gpt

## Load Data from Huggingface

In [8]:
#metadata = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full")
metadata = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Appliances", split="full")


Reusing dataset amazon_review2023 (/home/sagemaker-user/.cache/huggingface/datasets/McAuley-Lab___amazon_review2023/raw_meta_Appliances/0.0.0/16b76e0823d73bb8cff1e9c5e3e37dbc46ae3daee380417ae141f5e67d3ea8e8)


In [9]:
features_example = Features({'rating': Value(dtype='float'),
 'title': Value(dtype='string'),
 'text': Value(dtype='string'),
 'images': Sequence(feature=Value(dtype='string'), id=None),
 'asin':  Value(dtype='string'),
 'parent_asin':  Value(dtype='string'),
 'user_id': Value(dtype='string'),
 'timestamp': Value(dtype='int64'),
 'helpful_vote': Value(dtype='int32'),
 'verified_purchase': Value(dtype='bool')})

### Just use products with descriptions

In [10]:
#dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", features=features_example)
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Appliances", features=features_example)


Using custom data configuration raw_review_Appliances-8e916b816ac3c3e0
Reusing dataset amazon_review2023 (/home/sagemaker-user/.cache/huggingface/datasets/McAuley-Lab___amazon_review2023/raw_review_Appliances-8e916b816ac3c3e0/0.0.0/16b76e0823d73bb8cff1e9c5e3e37dbc46ae3daee380417ae141f5e67d3ea8e8)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
reviews_df = pd.DataFrame(dataset['full'][slice(0, 300000)])
products_df = pd.DataFrame(metadata[:300000])
df = reviews_df.merge(products_df, on='parent_asin')
df.shape

(300000, 25)

In [6]:
reviews_df[['parent_asin', 'title', 'text', 'rating']].to_csv('appliances_reviews.csv', index=False)

In [28]:
client = chromadb.PersistentClient(path="/home/sagemaker-user/user-default-efs/projects/persuasio/persuasio/chroma_small")
client2 = chromadb.PersistentClient(path="/home/sagemaker-user/user-default-efs/projects/persuasio/persuasio/chroma_small2")
#client3 = chromadb.PersistentClient(path="/home/sagemaker-user/user-default-efs/projects/persuasio/persuasio/chroma_appliances3")


In [13]:
with_description_df = df[[True  if len(x) > 0 else False for x in df['description']]]
descriptions = with_description_df[['asin', 'parent_asin', 'description', 'average_rating', 
                                    'title_y', 'price', 'rating_number', 'images_y']].drop_duplicates(subset='parent_asin', inplace=False)
descriptions.shape

(25005, 8)

### Vector DB

In [50]:
description_db = client.get_collection(name="amazon_beauty_descriptions")
description_db2 = client2.get_collection(name="amazon_beauty_descriptions2")
#description_db3 = client3.get_collection(name="amazon_appliances_descriptions3")
#print(description_db.count(), description_db2.count(), description_db3.count())


In [52]:
start = 5000
end = 10000
description_db2.add(
    documents=[x[0] for x in descriptions['description'].iloc[start:end]],
    ids=[str(x) for x in descriptions['parent_asin'].iloc[start:end]], # parent_asin makes it easier for now
    metadatas=[{'average_rating': r,
                'price': p,
                'rating_number': c,
#                'hi_res': "\n".join(i['hi_res']),
                'large': "\n".join(i['large']),
                'thumb': "\n".join(i['thumb']),
                'title': t} for r, p, c, i, t in zip(descriptions['average_rating'].iloc[start:end],
                                                     descriptions['price'].iloc[start:end],
                                                     descriptions['rating_number'].iloc[start:end],
                                                     descriptions['images_y'].iloc[start:end],
                                                     descriptions['title_y'].iloc[start:end])] # could pull this later
) 

In [34]:
description_db.count()

0